In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df1= pd.read_csv("Desktop/emoint tweet/sadness-ratings-0to1.train.txt", delimiter='\t', header=None)
df1.columns = ['Id', 'tweet', 'emotion', 'score']

In [3]:
print(df1)


        Id                                              tweet  emotion  score
0    40000                      Depression sucks! #depression  sadness  0.958
1    40001            Feeling worthless as always #depression  sadness  0.958
2    40002                       Feeling worthless as always   sadness  0.958
3    40003  My #Fibromyalgia has been really bad lately wh...  sadness  0.946
4    40004  Im think ima lay in bed all day and sulk. Life...  sadness  0.934
..     ...                                                ...      ...    ...
781  40781  @VivienLloyd Thank you so much! Just home - st...  sadness  0.104
782  40782              Just put the winter duvet on ☃️❄️🌬☔️   sadness  0.104
783  40783  @SilkInSide @TommyJoeRatliff that's so pretty!...  sadness  0.088
784  40784  @BluesfestByron second artist announcement loo...  sadness  0.083
785  40785  I can literally eat creamy pesto pasta topped ...  sadness  0.083

[786 rows x 4 columns]


In [4]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji
import re

In [5]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df1['tweet'] = df1['tweet'].apply(preprocess_tweet)

In [6]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df1['tweet'] = df1['tweet'].apply(remove_mentions)


In [7]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df1['tweet'] = df1['tweet'].apply(clean_text)



In [8]:
df1

,Id,tweet,emotion,score
0,40000,depression sucks #depression,sadness,0.958
1,40001,feeling worthless as always #depression,sadness,0.958
2,40002,feeling worthless as always,sadness,0.958
3,40003,my #fibromyalgia has been really bad lately wh...,sadness,0.946
4,40004,im think ima lay in bed all day and sulk life ...,sadness,0.934
...,...,...,...,...
781,40781,thank you so much just home stunned but so hap...,sadness,0.104
782,40782,just put the winter duvet on,sadness,0.104
783,40783,thats so pretty i love the sky in the backgrou...,sadness,0.088
784,40784,second artist announcement looking good #blues...,sadness,0.083


In [9]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

In [10]:
df1['tweet'] = df1['tweet'].apply(convert_emoji)

In [11]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df1['tweet'] = df1['tweet'].apply(tokenize_tweets)

In [12]:
df1

,Id,tweet,emotion,score
0,40000,"[depression, sucks, #depression]",sadness,0.958
1,40001,"[feeling, worthless, as, always, #depression]",sadness,0.958
2,40002,"[feeling, worthless, as, always]",sadness,0.958
3,40003,"[my, #fibromyalgia, has, been, really, bad, la...",sadness,0.946
4,40004,"[im, think, ima, lay, in, bed, all, day, and, ...",sadness,0.934
...,...,...,...,...
781,40781,"[thank, you, so, much, just, home, stunned, bu...",sadness,0.104
782,40782,"[just, put, the, winter, duvet, on]",sadness,0.104
783,40783,"[thats, so, pretty, i, love, the, sky, in, the...",sadness,0.088
784,40784,"[second, artist, announcement, looking, good, ...",sadness,0.083


In [13]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]

def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

df1['tweet'] = df1['tweet'].apply(remove_stopwords)



In [14]:
df1


,Id,tweet,emotion,score
0,40000,"[depression, sucks, #depression]",sadness,0.958
1,40001,"[feeling, worthless, always, #depression]",sadness,0.958
2,40002,"[feeling, worthless, always]",sadness,0.958
3,40003,"[#fibromyalgia, been, really, bad, lately, whi...",sadness,0.946
4,40004,"[im, think, ima, lay, bed, day, sulk, life, hi...",sadness,0.934
...,...,...,...,...
781,40781,"[thank, home, stunned, happy, dont, think, sun...",sadness,0.104
782,40782,"[put, winter, duvet]",sadness,0.104
783,40783,"[thats, pretty, love, sky, background, purple,...",sadness,0.088
784,40784,"[second, artist, announcement, looking, good, ...",sadness,0.083


In [15]:
import torch
from transformers import BertModel, BertTokenizer

model_name='bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features

df1['features'] = df1['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
df1

,Id,tweet,emotion,score,features
0,40000,"[depression, sucks, #depression]",sadness,0.958,"[[-0.1279724, 0.41236815, 0.051530708, -0.1606..."
1,40001,"[feeling, worthless, always, #depression]",sadness,0.958,"[[-0.12421638, 0.25974998, -0.0661158, -0.3355..."
2,40002,"[feeling, worthless, always]",sadness,0.958,"[[-0.23523621, 0.27867872, -0.5598585, -0.2093..."
3,40003,"[#fibromyalgia, been, really, bad, lately, whi...",sadness,0.946,"[[-0.04965788, 0.24742632, -0.007569845, -0.58..."
4,40004,"[im, think, ima, lay, bed, day, sulk, life, hi...",sadness,0.934,"[[-0.04689168, 0.025870707, 0.33319533, -0.149..."
...,...,...,...,...,...
781,40781,"[thank, home, stunned, happy, dont, think, sun...",sadness,0.104,"[[-0.042937506, 0.24438974, 0.10452047, -0.057..."
782,40782,"[put, winter, duvet]",sadness,0.104,"[[-0.18927328, 0.033073332, -0.024956549, 0.17..."
783,40783,"[thats, pretty, love, sky, background, purple,...",sadness,0.088,"[[-0.39431044, -0.13764822, -0.2091385, -0.067..."
784,40784,"[second, artist, announcement, looking, good, ...",sadness,0.083,"[[-0.21641317, 0.026052019, -0.18617798, -0.03..."


In [17]:
df1.features[0].shape

(6, 768)

In [18]:

df1.head(10)

,Id,tweet,emotion,score,features
0,40000,"[depression, sucks, #depression]",sadness,0.958,"[[-0.1279724, 0.41236815, 0.051530708, -0.1606..."
1,40001,"[feeling, worthless, always, #depression]",sadness,0.958,"[[-0.12421638, 0.25974998, -0.0661158, -0.3355..."
2,40002,"[feeling, worthless, always]",sadness,0.958,"[[-0.23523621, 0.27867872, -0.5598585, -0.2093..."
3,40003,"[#fibromyalgia, been, really, bad, lately, whi...",sadness,0.946,"[[-0.04965788, 0.24742632, -0.007569845, -0.58..."
4,40004,"[im, think, ima, lay, bed, day, sulk, life, hi...",sadness,0.934,"[[-0.04689168, 0.025870707, 0.33319533, -0.149..."
5,40005,"[try, fail, dont, try, still, fail, #confused,...",sadness,0.917,"[[-0.30267218, 0.38377964, -0.13703564, 0.1354..."
6,40006,"[extreme, sadness]",sadness,0.917,"[[-0.166719, 0.14257, -0.1267908, 0.038275078,..."
7,40007,"[life, one, word, depressing]",sadness,0.917,"[[-0.030349113, 0.13385968, -0.20066914, 0.058..."
8,40008,"[panic, attacks, worst, feeling, really, sick,...",sadness,0.917,"[[-0.31141603, 0.07427356, 0.34359968, -0.3796..."
9,40009,"[feel, grim, ugly, atm]",sadness,0.896,"[[-0.04971684, 0.49060404, 0.027603516, -0.081..."


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features = df1['features'].tolist()
padded_features = pad_sequences(features, padding='post')
padded_df = df1.copy()
padded_df['features'] = padded_features.tolist()

In [20]:
X = np.stack(padded_df['features'])
print('Input feature shape:', X.shape)

Input feature shape: (786, 41, 768)


In [21]:
y = np.array(padded_df['score']) 
print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (786, 41, 768)
Output shape: (786,)


In [24]:
X = np.reshape(X, (786, 41 * 768)) 
y = np.reshape(y, (786,))

In [25]:
X.shape

(786, 31488)

In [26]:
Y=df1['score'].copy()

     

In [27]:
X.shape

(786, 31488)

In [28]:
max_sequence_length = len(max(X, key=len))


In [29]:
import tensorflow as tf

In [30]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(8096, input_shape=(31488,), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.1))
model.add(Dense(4048, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(2024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(1012, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(506, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='linear'))

early_stopping = EarlyStopping(monitor='mean_squared_error', patience=1, mode='min')
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8096)              254934944 
                                                                 
 dropout (Dropout)           (None, 8096)              0         
                                                                 
 dense_1 (Dense)             (None, 4048)              32776656  
                                                                 
 dropout_1 (Dropout)         (None, 4048)              0         
                                                                 
 dense_2 (Dense)             (None, 2024)              8195176   
                                                                 
 dropout_2 (Dropout)         (None, 2024)              0         
                                                                 
 dense_3 (Dense)             (None, 1012)              2

In [31]:
X.shape

(786, 31488)

In [32]:
df1['tweet'] = df1['tweet'].apply(lambda tokens: ' '.join(tokens))
array = df1['tweet'].values
tensor = tf.convert_to_tensor(array)

In [33]:
X.shape

(786, 31488)

In [34]:
y.shape

(786,)

In [35]:
history = model.fit(X, Y, batch_size=27, epochs=10, shuffle=True, verbose=1, callbacks=[early_stopping])

Epoch 1/10
30/30 [==============================] - 177s 5s/step - loss: 61.7288 - mean_squared_error: 2.4094
Epoch 2/10
30/30 [==============================] - 130s 4s/step - loss: 25.3057 - mean_squared_error: 0.0570
Epoch 3/10
30/30 [==============================] - 132s 4s/step - loss: 17.1011 - mean_squared_error: 0.0368
Epoch 4/10
30/30 [==============================] - 132s 4s/step - loss: 12.9627 - mean_squared_error: 0.0325
Epoch 5/10
30/30 [==============================] - 172s 6s/step - loss: 10.3477 - mean_squared_error: 0.0325


In [36]:
model.save('emointjoy.h5','/Home')

In [37]:
from keras.models import load_model

loaded_model = load_model('emointjoy.h5')

In [38]:
gold= pd.read_csv("Desktop/emoint tweet/sadness-ratings-0to1.dev.gold.txt", delimiter='\t', header=None)
gold.columns = ['Id', 'tweet', 'emotion', 'score']

In [39]:
gold

,Id,tweet,emotion,score
0,40786,@1johndes ball watching &amp; Rojo'd header wa...,sadness,0.583
1,40787,"A pessimist is someone who, when opportunity k...",sadness,0.188
2,40788,A .500 season is all I'm looking for at this p...,sadness,0.688
3,40789,"Stars, when you shine,\nYou know how I feel.\n...",sadness,0.292
4,40790,All I want to do is watch some netflix but I a...,sadness,0.667
...,...,...,...,...
69,40855,Common app just randomly logged me out as I wa...,sadness,0.833
70,40856,"I'd rather laugh with the rarest genius, in be...",sadness,0.688
71,40857,If you #invest in my new #film I will stop ask...,sadness,0.458
72,40858,"Just watched Django Unchained, Other people ma...",sadness,0.333


In [40]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
gold['tweet'] = gold['tweet'].apply(preprocess_tweet)

In [41]:
gold

,Id,tweet,emotion,score
0,40786,@1johndes ball watching &amp; rojo'd header wa...,sadness,0.583
1,40787,"a pessimist is someone who, when opportunity k...",sadness,0.188
2,40788,a .500 season is all i'm looking for at this p...,sadness,0.688
3,40789,"stars, when you shine,\nyou know how i feel.\n...",sadness,0.292
4,40790,all i want to do is watch some netflix but i a...,sadness,0.667
...,...,...,...,...
69,40855,common app just randomly logged me out as i wa...,sadness,0.833
70,40856,"i'd rather laugh with the rarest genius, in be...",sadness,0.688
71,40857,if you #invest in my new #film i will stop ask...,sadness,0.458
72,40858,"just watched django unchained, other people ma...",sadness,0.333


In [42]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

gold['tweet'] = gold['tweet'].apply(remove_mentions)
gold

,Id,tweet,emotion,score
0,40786,ball watching &amp; rojo'd header was equally...,sadness,0.583
1,40787,"a pessimist is someone who, when opportunity k...",sadness,0.188
2,40788,a .500 season is all i'm looking for at this p...,sadness,0.688
3,40789,"stars, when you shine,\nyou know how i feel.\n...",sadness,0.292
4,40790,all i want to do is watch some netflix but i a...,sadness,0.667
...,...,...,...,...
69,40855,common app just randomly logged me out as i wa...,sadness,0.833
70,40856,"i'd rather laugh with the rarest genius, in be...",sadness,0.688
71,40857,if you #invest in my new #film i will stop ask...,sadness,0.458
72,40858,"just watched django unchained, other people ma...",sadness,0.333


In [43]:
def clean_gold(gold):
    gold  = re.sub(r'[^\w\s#@]', '', gold ) 
    gold  = re.sub(r'\d+', '', gold)  
    gold  = re.sub(r'\s+', ' ', gold).strip()
    return gold

gold['tweet'] = gold['tweet'].apply(clean_gold)


In [44]:
gold

,Id,tweet,emotion,score
0,40786,ball watching amp rojod header was equally dre...,sadness,0.583
1,40787,a pessimist is someone who when opportunity kn...,sadness,0.188
2,40788,a season is all im looking for at this point #...,sadness,0.688
3,40789,stars when you shinenyou know how i feelnscent...,sadness,0.292
4,40790,all i want to do is watch some netflix but i a...,sadness,0.667
...,...,...,...,...
69,40855,common app just randomly logged me out as i wa...,sadness,0.833
70,40856,id rather laugh with the rarest genius in beau...,sadness,0.688
71,40857,if you #invest in my new #film i will stop ask...,sadness,0.458
72,40858,just watched django unchained other people may...,sadness,0.333


In [45]:
def convert_emoji(tweet):
    gold = emoji.demojize(tweet)
    return gold

gold['tweet'] = gold['tweet'].apply(convert_emoji)

In [46]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

gold['tweet'] = gold['tweet'].apply(tokenize_tweets)

In [47]:
gold

,Id,tweet,emotion,score
0,40786,"[ball, watching, amp, rojod, header, was, equa...",sadness,0.583
1,40787,"[a, pessimist, is, someone, who, when, opportu...",sadness,0.188
2,40788,"[a, season, is, all, im, looking, for, at, thi...",sadness,0.688
3,40789,"[stars, when, you, shinenyou, know, how, i, fe...",sadness,0.292
4,40790,"[all, i, want, to, do, is, watch, some, netfli...",sadness,0.667
...,...,...,...,...
69,40855,"[common, app, just, randomly, logged, me, out,...",sadness,0.833
70,40856,"[id, rather, laugh, with, the, rarest, genius,...",sadness,0.688
71,40857,"[if, you, #invest, in, my, new, #film, i, will...",sadness,0.458
72,40858,"[just, watched, django, unchained, other, peop...",sadness,0.333


In [48]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]


def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

gold['tweet'] = gold['tweet'].apply(remove_stopwords)



In [49]:
gold

,Id,tweet,emotion,score
0,40786,"[ball, watching, amp, rojod, header, equally, ...",sadness,0.583
1,40787,"[pessimist, someone, who, opportunity, knocks,...",sadness,0.188
2,40788,"[season, im, looking, point, #depressing, #roy...",sadness,0.688
3,40789,"[stars, shinenyou, know, feelnscent, pine, nyo...",sadness,0.292
4,40790,"[want, do, watch, netflix, stuck, class, #depr...",sadness,0.667
...,...,...,...,...
69,40855,"[common, app, randomly, logged, writing, last,...",sadness,0.833
70,40856,"[id, rather, laugh, rarest, genius, beautiful,...",sadness,0.688
71,40857,"[#invest, new, #film, stop, asking, invest, ne...",sadness,0.458
72,40858,"[watched, django, unchained, people, may, frow...",sadness,0.333


In [50]:
import torch
from transformers import BertModel, BertTokenizer


model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    return features


gold['features'] = gold['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
gold["features"][0].shape

(11, 768)

In [52]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
features =gold['features'].tolist()

padded_features = pad_sequences(features, padding='post')

padded_df = gold.copy()
padded_df['features'] = padded_features.tolist()

In [53]:
X_gold= np.stack(padded_df['features'])
print('Input feature shape:', X_gold.shape)

Input feature shape: (74, 31, 768)


In [54]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 41

padded_gold_data = pad_sequences(X_gold, maxlen=max_sequence_length, padding='post', truncating='post')


In [55]:
X = np.reshape(padded_gold_data, (74, 41 * 768))  


In [56]:
gold_features = np.array(X)

In [57]:
print(gold_features.shape)

(74, 31488)


In [58]:

predictions = loaded_model.predict(gold_features)

3/3 [==============================] - 2s 205ms/step


In [59]:
print(predictions)

[[0.549318  ]
 [0.6317027 ]
 [0.5988664 ]
 [0.6064731 ]
 [0.6636544 ]
 [0.50672907]
 [0.69371194]
 [0.8059905 ]
 [0.68053985]
 [0.5170404 ]
 [0.55427194]
 [0.6421241 ]
 [0.67412335]
 [0.6315049 ]
 [0.5972324 ]
 [0.6229547 ]
 [0.57453316]
 [0.5955164 ]
 [0.6481537 ]
 [0.7584966 ]
 [0.5808796 ]
 [0.54725224]
 [0.59797645]
 [0.53159624]
 [0.5267597 ]
 [0.58217865]
 [0.5494032 ]
 [0.57955605]
 [0.6439913 ]
 [0.5330556 ]
 [0.62816125]
 [0.66092783]
 [0.6775344 ]
 [0.5956678 ]
 [0.48569632]
 [0.6963353 ]
 [0.67585987]
 [0.6221743 ]
 [0.6109994 ]
 [0.5300727 ]
 [0.58055323]
 [0.6518491 ]
 [0.68427366]
 [0.60066354]
 [0.6063927 ]
 [0.6304117 ]
 [0.6859485 ]
 [0.548276  ]
 [0.5710445 ]
 [0.5239772 ]
 [0.64430135]
 [0.60592365]
 [0.58504313]
 [0.6392455 ]
 [0.6416382 ]
 [0.6799915 ]
 [0.6645092 ]
 [0.75164324]
 [0.6676081 ]
 [0.6212203 ]
 [0.58799905]
 [0.5140269 ]
 [0.59214   ]
 [0.65871555]
 [0.5545823 ]
 [0.6325733 ]
 [0.60864407]
 [0.5718974 ]
 [0.48917645]
 [0.71098256]
 [0.6838189 ]
 [0.63

In [60]:
gold['prediction']=pd.DataFrame(predictions)

In [61]:
gold

,Id,tweet,emotion,score,features,prediction
0,40786,"[ball, watching, amp, rojod, header, equally, ...",sadness,0.583,"[[-0.43671992, 0.057505418, -0.19090301, -0.25...",0.549318
1,40787,"[pessimist, someone, who, opportunity, knocks,...",sadness,0.188,"[[0.0042859376, 0.30649495, 0.011315904, -0.31...",0.631703
2,40788,"[season, im, looking, point, #depressing, #roy...",sadness,0.688,"[[-0.36875078, 0.16265798, -0.020786183, -0.12...",0.598866
3,40789,"[stars, shinenyou, know, feelnscent, pine, nyo...",sadness,0.292,"[[-0.2151168, 0.26849976, -0.24530827, 0.24457...",0.606473
4,40790,"[want, do, watch, netflix, stuck, class, #depr...",sadness,0.667,"[[0.09977926, -0.09474957, 0.030625617, -0.020...",0.663654
...,...,...,...,...,...,...
69,40855,"[common, app, randomly, logged, writing, last,...",sadness,0.833,"[[-0.33207878, -0.17362261, -0.46206862, -0.11...",0.710983
70,40856,"[id, rather, laugh, rarest, genius, beautiful,...",sadness,0.688,"[[-0.17247936, 0.13550764, 0.11007807, -0.0802...",0.683819
71,40857,"[#invest, new, #film, stop, asking, invest, ne...",sadness,0.458,"[[0.06594571, 0.13042246, 0.050612476, -0.0706...",0.635404
72,40858,"[watched, django, unchained, people, may, frow...",sadness,0.333,"[[0.15952887, 0.13285294, 0.20451696, 0.080129...",0.559497


In [62]:
from sklearn.metrics import mean_squared_error

In [63]:
mse = mean_squared_error(gold['score'], gold['prediction'])
print("Mean square Error:", mse)


Mean square Error: 0.05000444392688266


In [65]:
test= pd.read_csv("Desktop/emoint tweet/sadness-ratings-0to1.test.target.txt", delimiter='\t', header=None)
test.columns = ['Id', 'tweet', 'emotion', 'score']

In [66]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
test['tweet'] = test['tweet'].apply(preprocess_tweet)

In [67]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

test['tweet'] = test['tweet'].apply(remove_mentions)
test

,Id,tweet,emotion,score
0,30902,you must be knowing #blithe means (adj.) happ...,joy,NONE
1,30903,old saying 'a #smile shared is one gained for ...,joy,NONE
2,30904,bridget jones' baby was bloody hilarious 😅 #br...,joy,NONE
3,30905,sparkling water makes your life sparkly,joy,NONE
4,30906,i'm tired of everybody telling me to chill out...,joy,NONE
...,...,...,...,...
709,31611,with a very tired body and mind and sparkling ...,joy,NONE
710,31612,i refuse to be a chirp chirp girl,joy,NONE
711,31613,it was very hard to stifle my laughter after i...,joy,NONE
712,31614,"while i was walking, a little boy in a red shi...",joy,NONE


In [68]:
def clean_test(test):
    test  = re.sub(r'[^\w\s#@]', '', test ) 
    test  = re.sub(r'\d+', '', test)  
    test  = re.sub(r'\s+', ' ', test).strip()
    return test

test['tweet'] = test['tweet'].apply(clean_test)


In [69]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

test['tweet'] = test['tweet'].apply(tokenize_tweets)

In [70]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'this', 'all',
    'am', 'or', 'but', 'if', 'my', 'me', 'we', 'us', 'our', 'we', 'up', 'down', 'out', 'just', 'how', 'why',
    'when', 'where', 'here', 'there', 'about', 'more', 'most', 'some', 'any', 'few', 'many', 'much', 'not',
    'only', 'other', 'same', 'such', 'no', 'nor', 'too', 'very', 'can', 'cannot', 'could', 'should', 'would',
    'might', 'must', 'shall', 'will', 'isn', 'hasn', 'doesn', 'haven', 'didn', 'hadn', 'wasn', 'weren',
    'wouldn', 'shouldn', 'ain', 'aren', 'ma'
]

def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens


test['tweet'] = test['tweet'].apply(remove_stopwords)



In [71]:
import torch
from transformers import BertModel, BertTokenizer


model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    return features


test['features'] = test['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [72]:
test["features"][0].shape

(12, 768)

In [73]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
features =test['features'].tolist()

padded_features = pad_sequences(features, padding='post')

padded_df = test.copy()
padded_df['features'] = padded_features.tolist()

In [74]:
X_test= np.stack(padded_df['features'])
print('Input feature shape:', X_test.shape)

Input feature shape: (714, 50, 768)


In [75]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = 41
truncated_test_data = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')

print(truncated_test_data.shape)

(714, 41, 768)


In [76]:
X = np.reshape(truncated_test_data, (714, 41 * 768)) 

In [77]:
test_features = np.array(X)

In [78]:
print(test_features.shape)

(714, 31488)


In [79]:

predictions = loaded_model.predict(test_features)

23/23 [==============================] - 4s 154ms/step


In [80]:
print(predictions)

[[0.5804511 ]
 [0.5894402 ]
 [0.6224335 ]
 [0.5385697 ]
 [0.708412  ]
 [0.63634473]
 [0.59076065]
 [0.6473655 ]
 [0.6540323 ]
 [0.61376566]
 [0.6210813 ]
 [0.655412  ]
 [0.50228876]
 [0.6055698 ]
 [0.55591184]
 [0.5460737 ]
 [0.6275031 ]
 [0.66009194]
 [0.5178999 ]
 [0.6467261 ]
 [0.5306929 ]
 [0.6505565 ]
 [0.6885378 ]
 [0.65472597]
 [0.5137574 ]
 [0.5696594 ]
 [0.60433674]
 [0.62825704]
 [0.7234407 ]
 [0.59075445]
 [0.60552377]
 [0.57617825]
 [0.7121013 ]
 [0.57952887]
 [0.5842244 ]
 [0.5308191 ]
 [0.5953137 ]
 [0.5893609 ]
 [0.72326285]
 [0.503528  ]
 [0.6392967 ]
 [0.64082557]
 [0.6600537 ]
 [0.62806803]
 [0.66868275]
 [0.6335969 ]
 [0.63893574]
 [0.63342696]
 [0.5933594 ]
 [0.5514099 ]
 [0.50270426]
 [0.7144894 ]
 [0.6790647 ]
 [0.6062217 ]
 [0.58606684]
 [0.63738936]
 [0.61701363]
 [0.5020289 ]
 [0.6103627 ]
 [0.484469  ]
 [0.5793373 ]
 [0.68889976]
 [0.61633235]
 [0.6382995 ]
 [0.49898547]
 [0.6044772 ]
 [0.5708076 ]
 [0.63988537]
 [0.582967  ]
 [0.6797536 ]
 [0.6090165 ]
 [0.62

In [81]:
test['score']=pd.DataFrame(predictions)

In [82]:
test

,Id,tweet,emotion,score,features
0,30902,"[knowing, #blithe, means, adj, happy, cheerful]",joy,0.580451,"[[0.06667158, 0.123148575, 0.20162007, -0.0633..."
1,30903,"[old, saying, #smile, shared, one, gained, ano...",joy,0.589440,"[[0.05411899, 0.09224143, 0.1278078, 0.0228617..."
2,30904,"[bridget, jones, baby, bloody, hilarious, #bri...",joy,0.622433,"[[-0.2968768, 0.16157933, 0.04914654, -0.04029..."
3,30905,"[sparkling, water, makes, life, sparkly]",joy,0.538570,"[[-0.025124839, 0.22698666, 0.04356417, 0.1450..."
4,30906,"[im, tired, everybody, telling, chill, everyth...",joy,0.708412,"[[-0.05045792, 0.482266, 0.18940437, -0.572387..."
...,...,...,...,...,...
709,31611,"[tired, body, mind, sparkling, teeth, say, fol...",joy,0.647323,"[[-0.0030458397, 0.1917828, 0.29441363, -0.136..."
710,31612,"[refuse, chirp, chirp, girl]",joy,0.540849,"[[-0.3924269, -0.31344494, -0.31823176, 0.0956..."
711,31613,"[hard, stifle, laughter, after, overheard, com...",joy,0.591366,"[[-0.042962182, 0.29848492, 0.035863742, 0.020..."
712,31614,"[while, walking, little, boy, red, shirt, year...",joy,0.513523,"[[-0.4283314, 0.22758897, -0.2881539, -0.34742..."


In [83]:
test=test.drop('features',axis=1)

In [84]:
test




,Id,tweet,emotion,score
0,30902,"[knowing, #blithe, means, adj, happy, cheerful]",joy,0.580451
1,30903,"[old, saying, #smile, shared, one, gained, ano...",joy,0.589440
2,30904,"[bridget, jones, baby, bloody, hilarious, #bri...",joy,0.622433
3,30905,"[sparkling, water, makes, life, sparkly]",joy,0.538570
4,30906,"[im, tired, everybody, telling, chill, everyth...",joy,0.708412
...,...,...,...,...
709,31611,"[tired, body, mind, sparkling, teeth, say, fol...",joy,0.647323
710,31612,"[refuse, chirp, chirp, girl]",joy,0.540849
711,31613,"[hard, stifle, laughter, after, overheard, com...",joy,0.591366
712,31614,"[while, walking, little, boy, red, shirt, year...",joy,0.513523
